# TAA PROJECT

## Imports

In [21]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)

# Import relevant libraries
import os
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from PIL import Image
import scipy.misc

import pandas as pd

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

## Load and Process Data

All data should be in the folder "data". Training data should be in the sub-folder "training_data" and test data in "test_data". The csv should be inside the data folder.

In [34]:
c = 0
X_train_orig = []
y_train_orig = []

X_test_orig = []
y_test_orig = []

path_train = "data/training_data/"
path_test = "data/test_data"

tmp1 = pd.read_csv("data/ISBI2016_ISIC_Part3_Training_GroundTruth.csv", header=None, names=['id', 'class'])
tmp2 = pd.read_csv("data/ISBI2016_ISIC_Part3_Test_GroundTruth.csv", header=None, names=['id', 'class'])

training_truth = { tmp1['id'][i]: tmp1['class'][i] for i in range(tmp1.shape[0])}
test_truth = { tmp2['id'][i]: tmp2['class'][i] for i in range(tmp2.shape[0])}

for filename in os.listdir(path_train):

    y_train_orig.append(training_truth[filename.rstrip('.jpg')])

    image = Image.open(path_train + filename)
    data =  np.asarray(image)
    #plt.figure(c)
    #imshow(data)
    data = image.resize((224,224), Image.BICUBIC)
    #print(data.size)
    data =  np.asarray(data)
    #plt.figure(c+100)
    #plt.imshow(data)
    #print(data.shape)
    X_train_orig.append(data) 
    #if c > 10:
    #    break
    c += 1

for filename in os.listdir(path_test):

    y_test_orig.append(test_truth[filename.rstrip('.jpg')])

    image = Image.open(path_test + filename)
    data =  np.asarray(image)
    data = image.resize((224,224), Image.BICUBIC)
    data =  np.asarray(data)
    X_test_orig.append(data) 

X_train_orig = np.array(X_train_orig)  
y_train_orig = np.array(y_train_orig)

X_test_orig = np.array(X_test_orig)  
y_test_orig = np.array(y_test_orig)
#print(X_train_orig.shape)
#print(y_train_orig.shape)


<bound method NDFrame.head of                id      class
0    ISIC_0000000     benign
1    ISIC_0000001     benign
2    ISIC_0000002  malignant
3    ISIC_0000004  malignant
4    ISIC_0000006     benign
..            ...        ...
895  ISIC_0011393     benign
896  ISIC_0011397     benign
897  ISIC_0011398     benign
898  ISIC_0011400     benign
899  ISIC_0011402     benign

[900 rows x 2 columns]> (900, 2)
ISIC_0000002 malignant
(900, 224, 224, 3)
(900,)
